In [34]:
# Imports


import requests
import zipfile
import io
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import pytz
import numpy as np
import math
import warnings
import seaborn as sns
import os
import shutil
import datetime

# Suppress specific warnings (in this case, FutureWarnings)
warnings.simplefilter(action='ignore', category=FutureWarning)

from sktime.clustering.k_medoids import TimeSeriesKMedoids
from scipy.fftpack import fft, fftfreq
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.metrics import silhouette_score, davies_bouldin_score
from tslearn.clustering import KShape
from scipy.signal import welch
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.cluster import DBSCAN
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

from scipy.stats import boxcox
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error

from tqdm.notebook import tqdm


In [35]:
def human_format(num):
  """Convert a number to a human readable version, e.g. 10M for 10,000,000."""
  if num == 0:
    return '0'
  letters = ["", "K", "M", "B", "T"]
  scale = math.floor(round(math.log(abs(num), 10)) / 3)
  # Convert to string: truncate to scale, remove trailing zeros and decimal points
  num = float('{:.3g}'.format(num)) / 10**(3*scale)
  num = str(num).rstrip('0').rstrip('.')
  # Append scale letter
  num += letters[scale]
  return num

def mpl_human_format(x, pos):
  if pos is not None:
    return human_format(x)

def make_axes_human_readable(ax, axis="y"):
  if axis.lower() == 'x':
    axisvar = ax.xaxis
  elif axis.lower() == 'y':
    axisvar = ax.yaxis
  else:
    raise("Must specify axis as either 'x' or 'y'")

  axisvar.set_major_formatter(matplotlib.ticker.FuncFormatter(mpl_human_format))
  return ax

In [36]:
# Load the electricity usage data
NUM_CLUSTERS = 4

cluster_dfs = list()

for i in range(1, NUM_CLUSTERS + 1):
    df = pd.concat([
        pd.read_parquet(f'../dataset/cluster_{i}/training.parquet'),
        pd.read_parquet(f'../dataset/cluster_{i}/validation.parquet'),
        pd.read_parquet(f'../dataset/cluster_{i}/test.parquet'),
    ], axis=1).T.sort_index()
    df.index = pd.to_datetime(df.index)
    df.index.name = 'date'
    cluster_dfs.append(df)

In [37]:
explanatory_variables_df = pd.read_parquet('../dataset/combined_explanatory_variables/explanatory_variables.parquet')
explanatory_variables_df.index = pd.to_datetime(explanatory_variables_df.index)
explanatory_variables_df = explanatory_variables_df.sort_index().asfreq('D')
explanatory_variables_df.head()

,heating_degree_days,cooling_degree_days,precip,precipprob,is_holiday,sunlight_length_hours,is_weekend
date,,,,,,,
2012-01-01,6.578559,0.0,0.000,0.0,0,10.0,1
2012-01-02,6.659918,0.0,0.111,1.0,0,10.0,0
2012-01-03,7.977833,0.0,0.000,0.0,0,10.0,0
2012-01-04,6.335269,0.0,0.000,0.0,0,10.0,0
2012-01-05,7.437771,0.0,0.000,0.0,0,10.0,0


In [70]:
melted_dfs = list()
for i, df in enumerate(cluster_dfs, start=1):
    melted_df = pd.melt(df.reset_index(), id_vars=['date'], var_name='customer', value_name='electricity_usage')
    melted_df['cluster'] = i
    melted_dfs.append(melted_df)

combined_melted_df = pd.concat(melted_dfs, axis=0)
customer_level_dataset_df = pd.merge(combined_melted_df, explanatory_variables_df, on='date', how='inner')
customer_level_dataset_df = customer_level_dataset_df.set_index(['date', 'customer', 'cluster'])
customer_level_dataset_df.head()
customer_level_dataset_df.shape

(289080, 8)

In [39]:
cutoff_date = pd.to_datetime('2014-01-01')

boxcox_dfs = list()
scaled_dfs = list()
mean_series = list()
mean_train_series = list()
mean_test_series = list()

for df in cluster_dfs:
    boxcox_df = df.apply(lambda column: boxcox(column)[0])
    scaled_df=boxcox_df
    #scaled_df = pd.DataFrame(
        #scaler.fit_transform(boxcox_df),
        #index=boxcox_df.index,
        #columns=boxcox_df.columns
    #)
    mean = scaled_df.mean(axis=1)
    mean_train = mean[mean.index < cutoff_date]
    mean_test = mean[mean.index >= cutoff_date]
    boxcox_dfs.append(boxcox_df)
    scaled_dfs.append(scaled_df)
    mean_series.append(mean)
    mean_train_series.append(mean_train)
    mean_test_series.append(mean_test)

In [72]:
cluster_num = 2

In [76]:
cluster_df = customer_level_dataset_df.xs(cluster_num, level='cluster')
cluster_df

,,electricity_usage,heating_degree_days,cooling_degree_days,precip,precipprob,is_holiday,sunlight_length_hours,is_weekend
date,customer,,,,,,,,
2012-01-02,MT_119,1547.207447,6.659918,0.0,0.111,1.0,0,10.0,0
2012-01-03,MT_119,1500.132979,7.977833,0.0,0.000,0.0,0,10.0,0
2012-01-04,MT_119,1622.872340,6.335269,0.0,0.000,0.0,0,10.0,0
2012-01-05,MT_119,1624.335106,7.437771,0.0,0.000,0.0,0,10.0,0
2012-01-06,MT_119,1635.638298,8.166583,0.0,0.000,0.0,0,10.0,0
...,...,...,...,...,...,...,...,...,...
2014-12-27,MT_082,1095.748547,9.032632,0.0,0.000,0.0,0,9.0,1
2014-12-28,MT_082,1063.953488,7.697900,0.0,0.000,0.0,0,9.0,1
2014-12-29,MT_082,1023.437500,10.921202,0.0,0.000,0.0,0,9.0,0


In [42]:
train_df = cluster_df[cluster_df.index.get_level_values('date') < cutoff_date]
test_df = cluster_df[cluster_df.index.get_level_values('date') >= cutoff_date]

In [43]:
corr = train_df.corr()
corr.style.background_gradient(cmap='coolwarm')

,electricity_usage,heating_degree_days,cooling_degree_days,precip,precipprob,is_holiday,sunlight_length_hours,is_weekend
electricity_usage,1.000000,-0.033898,0.035306,-0.007679,-0.014141,-0.003061,0.025987,0.000916
heating_degree_days,-0.033898,1.000000,-0.573343,0.087076,0.215837,0.030624,-0.684717,0.020313
cooling_degree_days,0.035306,-0.573343,1.000000,-0.159922,-0.285467,-0.050482,0.462142,-0.016394
precip,-0.007679,0.087076,-0.159922,1.000000,0.514216,0.083375,-0.161296,0.012477
precipprob,-0.014141,0.215837,-0.285467,0.514216,1.000000,0.068739,-0.227827,-0.012645
is_holiday,-0.003061,0.030624,-0.050482,0.083375,0.068739,1.000000,0.032473,0.005603
sunlight_length_hours,0.025987,-0.684717,0.462142,-0.161296,-0.227827,0.032473,1.000000,-0.001838
is_weekend,0.000916,0.020313,-0.016394,0.012477,-0.012645,0.005603,-0.001838,1.000000


In [44]:
highly_correlated = set()  # Use a set to store correlated columns
for i in range(len(corr.columns)):
  for j in range(i):
    if abs(corr.iloc[i, j]) > 0.8:
      print(corr.columns[i], corr.columns[j])
      colname = corr.columns[j]
      highly_correlated.add(colname)
train_df = train_df.drop(columns=highly_correlated)
test_df = test_df.drop(columns=highly_correlated)
train_df

,,electricity_usage,heating_degree_days,cooling_degree_days,precip,precipprob,is_holiday,sunlight_length_hours,is_weekend
date,customer,,,,,,,,
2012-01-02,MT_091,3141.260841,6.659918,0.0,0.111,1.0,0,10.0,0
2012-01-03,MT_091,3424.616411,7.977833,0.0,0.000,0.0,0,10.0,0
2012-01-04,MT_091,3465.810540,6.335269,0.0,0.000,0.0,0,10.0,0
2012-01-05,MT_091,3578.052035,7.437771,0.0,0.000,0.0,0,10.0,0
2012-01-06,MT_091,3697.965310,8.166583,0.0,0.000,0.0,0,10.0,0
...,...,...,...,...,...,...,...,...,...
2013-12-27,MT_362,863075.000000,5.549002,0.0,0.048,1.0,0,9.0,0
2013-12-28,MT_362,887050.000000,8.432045,0.0,0.101,1.0,0,9.0,1
2013-12-29,MT_362,889125.000000,9.526014,0.0,0.006,1.0,0,9.0,1


In [68]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
print(train_df)
# Assuming y_train is a 1D time series array for training.
# Adjust seasonal_periods and trend/seasonal types based on your data characteristics.
model = ExponentialSmoothing(train_df['customer'], 
                             trend='add',  # or 'mul' for multiplicative
                             seasonal='add', 
                             seasonal_periods=365) 

hw_fit = model.fit()

# Forecast next N steps (target_length in your LSTM)
forecast = hw_fit.forecast(steps=len(X_test))

                     electricity_usage  heating_degree_days  \
date       customer                                           
2012-01-02 MT_091          3141.260841             6.659918   
2012-01-03 MT_091          3424.616411             7.977833   
2012-01-04 MT_091          3465.810540             6.335269   
2012-01-05 MT_091          3578.052035             7.437771   
2012-01-06 MT_091          3697.965310             8.166583   
...                                ...                  ...   
2013-12-27 MT_362        863075.000000             5.549002   
2013-12-28 MT_362        887050.000000             8.432045   
2013-12-29 MT_362        889125.000000             9.526014   
2013-12-30 MT_362        844400.000000             9.994676   
2013-12-31 MT_362        874600.000000             6.318079   

                     cooling_degree_days  precip  precipprob  is_holiday  \
date       customer                                                        
2012-01-02 MT_091           

KeyError: 'customer'